In [1]:
from sqlalchemy import (
    create_engine, 
    MetaData,
    Table,
    Column,
    Integer,
    String,
    Numeric,
    Boolean,
    DateTime,
    ForeignKey
)
from datetime import datetime

In [2]:
# creating a schema

metadata = MetaData()

cookies = Table(
    "cookies",
    metadata,
    Column("cookie_id", Integer(), primary_key=True),
    Column("cookie_name", String(50), index=True),
    Column("cookie_recipe_url", String(255), nullable=False),
    Column("cookie_sku", String(55), nullable=False),
    Column("quantity", Integer()),
    Column("unit_cost", Numeric(12, 2))
)

users = Table(
    "users",
    metadata,
    Column("user_id", Integer(), primary_key=True),
    Column("username", String(50), unique=True, nullable=False),
    Column("email_address", String(255), nullable=False),
    Column("phone", String(20), nullable=False),
    Column("password", String(25), nullable=False),
    Column("created_on", DateTime(), default=datetime.now),
    Column("updated_on", DateTime, default=datetime.now, onupdate=datetime.now),
)

orders = Table(
    "orders",
    metadata,
    Column("order_id", Integer(), primary_key=True),
    Column("user_id", Integer(), ForeignKey("users.user_id")),
    Column("shipped", Boolean(), default=False),
)

line_items = Table(
    "line_items",
    metadata,
    Column("line_item_id", Integer(), primary_key=True),
    Column("order_id", ForeignKey("orders.order_id")),
    Column("cookie_id", ForeignKey("cookies.cookie_id")),
    Column("quantity", Integer()),
    Column("extended_cost", Numeric(12, 2))
)

# composite primary keys can be added by adding primary_key=True for more
# than one column

In [3]:
# commiting tables

engine = create_engine("sqlite:///:memory:")
metadata.create_all(engine)

In [4]:
# inserting into a database
from sqlalchemy.sql import insert, distinct


connection = engine.connect()

ins = cookies\
    .insert()\
    .values(
        cookie_name="chocolate chip",
        cookie_recipe_url="http://some.url",
        cookie_sku="CC01",
        unit_cost="0.05",
        quantity="12"  # numeric
        )

# print(str(ins))  # This shows the raw sql 
# print(ins.compile().params)  # this shows the escaped parameters

rp = connection.execute(ins)
# print(rp.inserted_primary_key)


ins = insert(cookies)\
        .values(
            cookie_name="dark chocolate chip",
            cookie_recipe_url="http://some2.url",
            cookie_sku="CC02",
            unit_cost="0.15",
            quantity="10"  # numeric
        )

rp = connection.execute(ins)
# print(rp.rowcount)

to_insert = [
    {
        "cookie_name": "peanut butter",
        "cookie_recipe_url": "http://some.website/peanut.com",
        "cookie_sku":"PB01",
        "quantity": "24",
        "unit_cost":"0.25"
    },
        {
        "cookie_name": "oatmeal raisin",
        "cookie_recipe_url": "http://some.website/raisin.com",
        "cookie_sku":"EWW01",
        "quantity": "24",
        "unit_cost":"0.25"
    }
]


rp = connection.execute(
        insert(cookies),
        to_insert
)
#print(rp.rowcount)
    

In [5]:
# Querying Data
from sqlalchemy.sql import select, desc, func

s = select([cookies])

rp = connection.execute(s)
results = rp.fetchall()
# for result in results:
#     print(result) 


s = cookies.select()
rp = connection.execute(s)
# rp.fetchall()
# first_row = rp.first()
# print(first_row[1])
# print(first_row.cookie_name)
# print(first_row[cookies.c.cookie_name])
# [r.cookie_name for r in rp]
# [col_name for col_name in rp.keys()] 


s = select([cookies.c.cookie_name, cookies.c.quantity])

rp = connection.execute(s)
# [r for r in rp]

s = select([cookies.c.cookie_name, cookies.c.quantity])\
        .order_by(cookies.c.quantity)

rp = connection.execute(s)
# [r for r in rp]

s = select([cookies.c.cookie_name, cookies.c.quantity])\
        .order_by(desc(cookies.c.quantity))

rp = connection.execute(s)
# [r for r in rp]


s = select([cookies.c.cookie_name, cookies.c.quantity])\
        .order_by(desc(cookies.c.quantity))\
        .limit(2)

rp = connection.execute(s)
# [r for r in rp]

s = select([func.sum(cookies.c.quantity).label('total')])

rp = connection.execute(s)
# rp.first().total

s = select([func.count(cookies.c.cookie_name)])

rp = connection.execute(s)
# rp.scalar()

/home/rory/repos/sql_book/venv/lib/python3.7/site-packages/sqlalchemy/sql/sqltypes.py:666: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


In [6]:
# Filtering

from sqlalchemy.sql import and_, or_, not_
from sqlalchemy import distinct, cast 

s = select([cookies])\
        .where(cookies.c.cookie_name == "chocolate chip")

rp = connection.execute(s)
# [r for r in rp]

s = select([cookies])\
    .where(cookies.c.cookie_name.like("%chocolate%"))

rp = connection.execute(s)
# print([r for r in rp])

s = select([distinct(cookies.c.cookie_name)])
rp = connection.execute(s)
# [r for r in rp]


s = select([cookies])\
        .where(not_(cookies.c.cookie_name.like("%chocolate%")))
rp = connection.execute(s)
# print([r for r in rp])

s = select([cookies.c.cookie_name, "SKU_" + cookies.c.cookie_sku])
rp = connection.execute(s)
# print([r for r in rp])

s = select([cookies.c.cookie_name,
            cast((cookies.c.unit_cost * cookies.c.quantity), Numeric(12, 2))\
                .label("inv_cost")
           ])
rp = connection.execute(s)
# print([r for r in rp])

s = select([cookies])\
        .where(
            and_(
                cookies.c.quantity > 3,
                cookies.c.unit_cost > 0.1
                )
        )
rp = connection.execute(s)
# print([r for r in rp])

In [7]:
# Updating Data

from sqlalchemy.sql import update 

u = update(cookies)\
        .where(cookies.c.cookie_name == "chocolate chip")\
        .values(quantity=(cookies.c.quantity + 120))
rp = connection.execute(u)
# print(rp.rowcount)


In [8]:
# Deleting Data

from sqlalchemy.sql import delete 

d = delete(cookies)\
        .where(cookies.c.cookie_name == "dark chocolate chip")

rp = connection.execute(d)
rp.rowcount

1

In [9]:
# Loading data


customer_list = [
    {
        "username": "cookiemon",
        "email_address": "man@cookie.com",
        "phone": "111-111-1111",
        "password": "password"
    },
        {
        "username": "cakeeater",
        "email_address": "cakeearter@cake.com",
        "phone": "222-222-2222",
        "password": "password"
    },
        {
        "username": "pieguy",
        "email_address": "pie@guy.com",
        "phone": "333-333-3333",
        "password": "password"
    }
]
ins = insert(users)
rp = connection.execute(ins, customer_list)


ins = insert(orders)\
        .values(order_id=1, user_id=1)
result=connection.execute(ins)

order_items = [
    {
        "order_id": 1,
        "cookie_id": 1,
        "quantity": 2,
        "extended_cost": 1.00
    },
        {
        "order_id": 1,
        "cookie_id": 3,
        "quantity": 12,
        "extended_cost": 3.00
    },
]
ins = insert(line_items)
rp = connection.execute(ins, order_items)

ins = insert(orders).values(user_id=2, order_id=2)
rp = connection.execute(ins)

order_items = [
    {
        "order_id": 2,
        "cookie_id": 1,
        "quantity": 24,
        "extended_cost": 12.00
    },
        {
        "order_id": 2,
        "cookie_id": 4,
        "quantity": 6,
        "extended_cost": 6.00
    },
]
ins = insert(line_items)
rp = connection.execute(ins, order_items)


In [10]:
# JOINS

s = select([
        orders.c.order_id, 
        users.c.username, 
        users.c.phone, 
        cookies.c.cookie_name, 
        line_items.c.quantity, 
        line_items.c.extended_cost
        ])\
            .select_from(
                orders\
                    .join(users)\
                    .join(line_items)\
                    .join(cookies)
            )\
            .where(
                users.c.username == "cookiemon"
            )
rp = connection.execute(s)
# print([r for r in rp])

s = select(
        [
            users.c.username,
            func.count(orders.c.order_id)
        ]
        )\
    .select_from(users.outerjoin(orders))\
    .group_by(users.c.username)

rp = connection.execute(s)
print([r for r in rp])


[('cakeeater', 1), ('cookiemon', 1), ('pieguy', 0)]


In [11]:
# Grouping

s = select([users.c.username, func.count(orders.c.order_id)])\
        .select_from(users.join(orders))\
        .group_by(users.c.username)

rp = connection.execute(s)
# print([r for r in rp])



In [23]:
# Chaining

def get_orders_by_customer(customer_name):
    s = select([orders.c.order_id, users.c.username, 
                users.c.phone, cookies.c.cookie_name, 
                line_items.c.quantity, line_items.c.extended_cost
                ])\
                    .select_from(orders.join(users).join(line_items).join(cookies))\
                    .where(users.c.username==customer_name)
    rp = connection.execute(s)
    results = rp.fetchall()
    return results 

# print(get_orders_by_customer("cookiemon"))

def get_orders_by_customer(customer_name, shipped=None, details=False):
    columns = [orders.c.order_id, users.c.username, users.c.phone]
    joins = users.join(orders)
    if details:
        columns.extend([cookies.c.cookie_name, line_items.c.quantity, line_items.c.extended_cost])
        joins = joins.join(line_items).join(cookies)
    
    s = select(columns).select_from(joins)\
                        .where(users.c.username == customer_name)
    if shipped is not None:
        s = s.where(orders.c.shipped == shipped)
    result = connection.execute(s)
    return result.fetchall()

# get_orders_by_customer("cookiemon", details=True, shipped=False)            

In [26]:
# Raw Queries

from sqlalchemy.sql import text

rp = connection.execute("SELECT * FROM cookies")
# print([r for r in rp])

s = select([cookies])\
        .where(text("cookie_name = 'chocolate chip'"))
rp = connection.execute(s)
# print([r for r in rp])



[(1, 'chocolate chip', 'http://some.url', 'CC01', 132, Decimal('0.05'))]
